In [2]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

Preparing data

In [3]:
from src.read_file_to_df import read_file_to_df

df1 = read_file_to_df('../data/spell-testset1.txt')
df2 = read_file_to_df('../data/spell-testset2.txt')

# ===== Misleading incorrect string ===== #
df3 = read_file_to_df('../data/aspell.txt')
df4 = read_file_to_df('../data/birkbeck.txt')
df5 = read_file_to_df('../data/wikipedia.txt')

In [4]:
df_combined = pd.concat([df1, df2], axis=0, ignore_index=True)

In [5]:
df_combined.to_csv('../data/combined.csv', index=False)

In [6]:
df_combined = pd.read_csv('../data/combined.csv')

In [7]:
df_sampled = df_combined.sample(n=100, random_state=42)

[PySpellChecker](https://pypi.org/project/pyspellchecker/ - Levenshtein distance

In [8]:
from spellchecker import SpellChecker
from tqdm import tqdm

spell = SpellChecker()

def get_correction(word):
    misspelled = spell.unknown([word])
    if misspelled:
        correction = spell.correction(word)
        if correction:
            return correction
        else:
            return "incorrectWord"
    return word

tqdm.pandas()

df_sampled['py_spell_checker'] = df_sampled['incorrect'].progress_apply(get_correction)


100%|██████████| 100/100 [00:13<00:00,  7.41it/s][A


In [9]:
y_true = df_sampled['correct']
y_pred = df_sampled['py_spell_checker']

In [10]:
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='macro', zero_division=0)
recall = recall_score(y_true, y_pred, average='macro', zero_division=0)
f1 = f1_score(y_true, y_pred, average='macro')

In [11]:
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

Accuracy: 0.73
Precision: 0.5855855855855856
Recall: 0.5780780780780781
F1 Score: 0.5807807807807808


In [12]:
with open('../data/py_spellchecker_metrics.txt', 'w') as f:
    f.write(f'Accuracy: {accuracy}\n')
    f.write(f'Precision: {precision}\n')
    f.write(f'Recall: {recall}\n')
    f.write(f'F1 Score: {f1}\n')

[Bhuvana/t5-base-spellchecker](https://huggingface.co/Bhuvana/t5-base-spellchecker)

In [13]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
tokenizer = T5Tokenizer.from_pretrained("Bhuvana/t5-base-spellchecker")

model = T5ForConditionalGeneration.from_pretrained("Bhuvana/t5-base-spellchecker")

In [14]:
def correct(inputs):
    input_ids = tokenizer.encode(inputs,return_tensors='pt')
    sample_output = model.generate(
        input_ids,
        do_sample=True,
        max_length=50,
        top_p=0.99,
        num_return_sequences=1
    )
    res = tokenizer.decode(sample_output[0], skip_special_tokens=True)
    return res

In [15]:
tqdm.pandas()

df_sampled['base_spellchecker'] = df_sampled['incorrect'].progress_apply(correct)


100%|██████████| 100/100 [00:35<00:00,  2.81it/s]


In [19]:
y_true = df_sampled['correct']
y_pred = df_sampled['base_spellchecker']

In [20]:
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='macro', zero_division=0)
recall = recall_score(y_true, y_pred, average='macro', zero_division=0)
f1 = f1_score(y_true, y_pred, average='macro')

In [21]:
with open('../data/base_spellchecker_metrics.txt', 'w') as f:
    f.write(f'Accuracy: {accuracy}\n')
    f.write(f'Precision: {precision}\n')
    f.write(f'Recall: {recall}\n')
    f.write(f'F1 Score: {f1}\n')

In [22]:
# Load metrics and comapre them in dataframe
with open('../data/py_spellchecker_metrics.txt', 'r') as f:
    py_spellchecker_metrics = f.readlines()
    
with open('../data/base_spellchecker_metrics.txt', 'r') as f:
    base_spellchecker_metrics = f.readlines()
    
py_spellchecker_metrics = [metric.strip() for metric in py_spellchecker_metrics]
base_spellchecker_metrics = [metric.strip() for metric in base_spellchecker_metrics]

df_metrics = pd.DataFrame({
    'py_spellchecker': py_spellchecker_metrics,
    'base_spellchecker': base_spellchecker_metrics
})

In [23]:
df_metrics

,py_spellchecker,base_spellchecker
0,Accuracy: 0.73,Accuracy: 0.09
1,Precision: 0.5855855855855856,Precision: 0.05
2,Recall: 0.5780780780780781,Recall: 0.04351851851851852
3,F1 Score: 0.5807807807807808,F1 Score: 0.045370370370370366
